In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [ ]:
registry_patient_selection_path = '/Users/jk1/temp/opsum_extraction_output/high_frequency_data_patient_selection_with_details.csv'
ehr_extraction_path = '/Users/jk1/stroke_datasets/stroke_unit_dataset/per_value/Extraction_20221117'
output_path = '/Users/jk1/temp/opsum_extraction_output/ehr_extraction_errors/Extraction_20221117'

In [ ]:
registry_data_df = pd.read_csv(registry_patient_selection_path, dtype=str)

In [ ]:
registry_data_df

In [ ]:
registry_data_df['Arrival time'].apply(lambda x: ':'.join(x.split(':')[:2]))

In [ ]:
registry_data_df['registry_stroke_onset'] = pd.to_datetime(pd.to_datetime(registry_data_df['Stroke onset date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
                                        + ' ' + registry_data_df['Stroke onset time'].apply(lambda x: ':'.join(str(x).split(':')[:2])), format='%d-%m-%Y %H:%M').dt.strftime('%d.%m.%Y %H:%M')

registry_data_df['registry_admission_date'] = pd.to_datetime(pd.to_datetime(registry_data_df['Arrival at hospital'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
                                        + ' ' + registry_data_df['Arrival time'].apply(lambda x: ':'.join(str(x).split(':')[:2])), format='%d-%m-%Y %H:%M').dt.strftime('%d.%m.%Y %H:%M')

In [ ]:
from preprocessing.utils import create_registry_case_identification_column

registry_data_df['case_admission_id'] = create_registry_case_identification_column(registry_data_df)

In [ ]:
len(registry_data_df['case_admission_id'].unique())

In [ ]:
import pandas as pd
eds_df = pd.read_csv(os.path.join(ehr_extraction_path,'eds_j1.csv'), delimiter=';', dtype=str)

In [ ]:
from preprocessing.utils import create_ehr_case_identification_column

eds_df['case_admission_id'] = create_ehr_case_identification_column(eds_df)

In [ ]:
from preprocessing.patient_selection.filter_ehr_patients import filter_ehr_patients

eds_df = filter_ehr_patients(eds_df, registry_patient_selection_path)


In [ ]:
len(eds_df['case_admission_id'].unique())

In [ ]:
eds_df[eds_df.patient_id == '97738228'][['patient_id', 'patient_id_manual', 'eds_final_patient_id', 'eds_end_4digit', 'eds_manual', 'eds_final_id']]

Restrict to patient selection

In [ ]:
restricted_eds_df = eds_df[eds_df['case_admission_id'].isin(registry_data_df['case_admission_id'])]


In [ ]:
joined_df = pd.merge(registry_data_df, restricted_eds_df, on=['case_admission_id'], how='left')

## Find patients with differing admission dates

Difference between stroke registry admission (registry_admission_date) and DPI admission (eds_final_begin)

In [ ]:
joined_df['registry_arrival_to_eds_begin_days'] = (pd.to_datetime(joined_df['eds_final_begin'], format='%d.%m.%Y %H:%M') - pd.to_datetime(joined_df['Arrival at hospital'], format='%Y%m%d')).dt.days


Find patients with a difference of over 10 days in admission date (thus 14 days period does not cover 4 days of admission)

In [ ]:
differing_starting_date_df = joined_df[(joined_df['registry_arrival_to_eds_begin_days'] > 0) |(joined_df['registry_arrival_to_eds_begin_days'] < -10)]

In [ ]:
differing_starting_date_df[['case_admission_id', 'patient_id_x', 'patient_id_manual', 'eds_final_patient_id', 'eds_final_begin', 'Arrival at hospital', 'registry_arrival_to_eds_begin_days', 'registry_stroke_onset', 'eds_final_end', 'date_from']]

In [ ]:
print('N patients with differing admission dates')
differing_starting_date_df.case_admission_id.nunique()

In [ ]:
differing_starting_date_df.drop(['Unnamed: 0', 'Stroke onset date', 'Stroke onset time', 'nr', 'patient_id_y',
       'EDS_last_4_digits', 'DOB_y', 'begin_date', 'end_date',
       'death_date', 'death_hosp', 'eds_final_id', 'eds_final_begin',
       'eds_final_end', 'eds_final_patient_id', 'eds_final_birth', 'Referral_y',
       'eds_final_death', 'eds_final_birth_str', 'date_from', 'date_to', 'eds_manual'], axis=1, inplace=True)

In [ ]:
differing_starting_date_df.to_csv(os.path.join(output_path, 'patients_with_wrong_eds_start.csv'), index=False)

## Find patients with difference in stroke onset and start of extraction time

Find difference between registry stroke onset and date_from

In [ ]:
joined_df['registry_onset_to_date_from_days'] = (pd.to_datetime(joined_df['date_from'], format='%d.%m.%Y %H:%M') - pd.to_datetime(joined_df['registry_stroke_onset'], format='%d.%m.%Y %H:%M')).dt.days

In [ ]:
joined_df[(joined_df['registry_onset_to_date_from_days'] <  -2) |(joined_df['registry_onset_to_date_from_days'] > 0)]